# Creating Custom Networks for Multi-Class Classification

This tutorial demonstrates how to define, train, and use different models for multi-class classification. We will reuse most of the code from the [Logistic Regression](Logistic_Regression.html) tutorial so if you haven't gone through that, consider reviewing it first. 

## Import Modules

We start by importing the modules we will use in our code.

In [ ]:
%pip --quiet install objax

import os

import numpy as np
import tensorflow_datasets as tfds

import objax
from objax.util import EasyDict
from objax.zoo.dnnet import DNNet

## Load the data

Next, we will load the "[MNIST](http://yann.lecun.com/exdb/mnist/)" dataset from [TensorFlow DataSets](https://www.tensorflow.org/datasets/api_docs/python/tfds). This dataset contains handwritten digits (i.e., numbers between 0 and 9) and to correctly identify each handwritten digit. 

The ``prepare`` method pads 2 pixels to the left, right, top, and bottom of each image to resize into 32 x 32 pixes. While MNIST images are grayscale the 'prepare` method expands each image to three color channels to demonstrate the process of working with color images. The same method also rescales each pixel value to [-1, 1], and converts the image to (N, C, H, W) format.

In [ ]:
# Data: train has 60000 images - test has 10000 images
# Each image is resized and converted to 32 x 32 x 3
DATA_DIR = os.path.join(os.environ['HOME'], 'TFDS')
data = tfds.as_numpy(tfds.load(name='mnist', batch_size=-1, data_dir=DATA_DIR))

def prepare(x):
  """Pads 2 pixels to the left, right, top, and bottom of each image, scales pixel value to [-1, 1], and converts to NCHW format."""
  s = x.shape
  x_pad = np.zeros((s[0], 32, 32, 1))
  x_pad[:, 2:-2, 2:-2, :] = x
  return objax.util.image.nchw(
      np.concatenate([x_pad.astype('f') * (1 / 127.5) - 1] * 3, axis=-1))

train = EasyDict(image=prepare(data['train']['image']), label=data['train']['label'])
test = EasyDict(image=prepare(data['test']['image']), label=data['test']['label'])
ndim = train.image.shape[-1]

del data

## Deep Neural Network Model

Objax offers many predefined models that we can use for classification. One example is the ``objax.zoo.DNNet`` model comprising multiple fully connected layers with configurable size and activation functions. 

In [ ]:
dnn_layer_sizes = 3072, 128, 10
dnn_model = DNNet(dnn_layer_sizes, objax.functional.leaky_relu)

## Custom Model Definition

Alternatively, we can define a new model customized to our machine learning task. We demonstrate this process by defining a convolutional network (ConvNet) from scratch. 

We use ``objax.nn.Sequential`` to compose multiple layers of convolution (``objax.nn.Conv2D``), batch normalization (``objax.nn.BatchNorm2D``), ReLU (``objax.functional.relu``), Max Pooling (``objax.functional.max_pool_2d``), Average Pooling (``jax.mean``), and Linear (``objax.nn.Linear``) layers.

Since [batch normalization layer](https://arxiv.org/abs/1502.03167) behaves differently at training and at prediction, we pass the ``training`` flag to a ``__call__`` function of ``ConvNet`` class. We also use the flag to output logits at training and probability at prediction.

In [ ]:
class ConvNet(objax.Module):
  """ConvNet implementation."""

  def __init__(self, nin, nclass):
    """Define 3 blocks of conv-bn-relu-conv-bn-relu followed by linear layer."""
    self.conv_block1 = objax.nn.Sequential([objax.nn.Conv2D(nin, 16, 3, use_bias=False),
                                            objax.nn.BatchNorm2D(16),
                                            objax.functional.relu,
                                            objax.nn.Conv2D(16, 16, 3, use_bias=False),
                                            objax.nn.BatchNorm2D(16),
                                            objax.functional.relu])
    self.conv_block2 = objax.nn.Sequential([objax.nn.Conv2D(16, 32, 3, use_bias=False),
                                            objax.nn.BatchNorm2D(32),
                                            objax.functional.relu,
                                            objax.nn.Conv2D(32, 32, 3, use_bias=False),
                                            objax.nn.BatchNorm2D(32),
                                            objax.functional.relu])
    self.conv_block3 = objax.nn.Sequential([objax.nn.Conv2D(32, 64, 3, use_bias=False),
                                            objax.nn.BatchNorm2D(64),
                                            objax.functional.relu,
                                            objax.nn.Conv2D(64, 64, 3, use_bias=False),
                                            objax.nn.BatchNorm2D(64),
                                            objax.functional.relu])
    self.linear = objax.nn.Linear(64, nclass)

  def __call__(self, x, training):
    x = self.conv_block1(x, training=training)
    x = objax.functional.max_pool_2d(x, size=2, strides=2)
    x = self.conv_block2(x, training=training)
    x = objax.functional.max_pool_2d(x, size=2, strides=2)
    x = self.conv_block3(x, training=training)
    x = x.mean((2, 3))
    x = self.linear(x)
    return x
    

In [ ]:
cnn_model = ConvNet(nin=3, nclass=10)
cnn_model.vars().print()

(ConvNet).conv_block1(Sequential)[0](Conv2D).w                    432 (3, 3, 3, 16)
(ConvNet).conv_block1(Sequential)[1](BatchNorm2D).running_mean       16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[1](BatchNorm2D).running_var       16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[1](BatchNorm2D).beta             16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[1](BatchNorm2D).gamma            16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[3](Conv2D).w                   2304 (3, 3, 16, 16)
(ConvNet).conv_block1(Sequential)[4](BatchNorm2D).running_mean       16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[4](BatchNorm2D).running_var       16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[4](BatchNorm2D).beta             16 (1, 16, 1, 1)
(ConvNet).conv_block1(Sequential)[4](BatchNorm2D).gamma            16 (1, 16, 1, 1)
(ConvNet).conv_block2(Sequential)[0](Conv2D).w                   4608 (3, 3, 16, 32)
(ConvNet).conv_block2(Sequential)[1](BatchNorm2D).running_mean      

## Model Training and Evaluation

The ``train_model`` method combines all the parts of defining the loss function, gradient descent, training loop, and evaluation. It takes the ``model`` as a parameter so it can be reused with the two models we defined earlier.

Unlike the Logistic Regression tutorial we use the ``objax.functional.loss.cross_entropy_logits_sparse`` because we perform multi-class classification. The optimizer, gradient descent operation, and training loop remain the same. 

The ``DNNet`` model expects flattened images whereas ``ConvNet`` images in (C, H, W) format. The ``flatten_image`` method prepares images before passing them to the model. 

When using the model for inference we apply the ``objax.functional.softmax`` method to compute the probability distribution from the model's logits. 

In [ ]:
# Settings
lr = 0.03  # learning rate
batch = 128
epochs = 100

# Train loop

def train_model(model):

  def predict(model, x):
    """""" 
    return objax.functional.softmax(model(x,  training=False))
    
  def flatten_image(x):
    """Flatten the image before passing it to the DNN."""
    if isinstance(model, DNNet):
      return objax.functional.flatten(x)
    else:
      return x
  
  opt = objax.optimizer.Momentum(model.vars())

  # Cross Entropy Loss
  def loss(x, label):
    return objax.functional.loss.cross_entropy_logits_sparse(model(x, training=True), label).mean()

  gv = objax.GradValues(loss, model.vars())
  def train_op(x, label):
    g, v = gv(x, label)  # returns gradients, loss
    opt(lr, g)
    return v

  train_op = objax.Jit(train_op, gv.vars() + opt.vars())  
  
  for epoch in range(epochs):
    avg_loss = 0
    # randomly shuffle training data
    shuffle_idx = np.random.permutation(train.image.shape[0])
    for it in range(0, train.image.shape[0], batch):
      sel = shuffle_idx[it: it + batch]
      avg_loss += float(train_op(flatten_image(train.image[sel]), train.label[sel])[0]) * len(sel)
    avg_loss /= it + len(sel)

    # Eval
    accuracy = 0
    for it in range(0, test.image.shape[0], batch):
      x, y = test.image[it: it + batch], test.label[it: it + batch]
      accuracy += (np.argmax(predict(model, flatten_image(x)), axis=1) == y).sum()  
    accuracy /= test.image.shape[0]
    print('Epoch %04d  Loss %.2f  Accuracy %.2f' % (epoch + 1, avg_loss, 100 * accuracy))

## Training the DNN Model

In [ ]:
train_model(dnn_model)

Epoch 0001  Loss 2.85  Accuracy 11.35
Epoch 0002  Loss 2.16  Accuracy 32.86
Epoch 0003  Loss 1.63  Accuracy 47.15
Epoch 0004  Loss 1.31  Accuracy 52.86
Epoch 0005  Loss 1.21  Accuracy 55.17
Epoch 0006  Loss 1.14  Accuracy 58.27
Epoch 0007  Loss 1.09  Accuracy 63.68
Epoch 0008  Loss 0.97  Accuracy 71.96
Epoch 0009  Loss 0.83  Accuracy 74.87
Epoch 0010  Loss 0.75  Accuracy 76.94
Epoch 0011  Loss 0.73  Accuracy 78.59
Epoch 0012  Loss 0.69  Accuracy 79.81
Epoch 0013  Loss 0.65  Accuracy 81.46
Epoch 0014  Loss 0.61  Accuracy 82.58
Epoch 0015  Loss 0.58  Accuracy 83.11
Epoch 0016  Loss 0.57  Accuracy 83.69
Epoch 0017  Loss 0.55  Accuracy 84.09
Epoch 0018  Loss 0.54  Accuracy 84.36
Epoch 0019  Loss 0.52  Accuracy 85.24
Epoch 0020  Loss 0.50  Accuracy 86.27
Epoch 0021  Loss 0.47  Accuracy 86.69
Epoch 0022  Loss 0.46  Accuracy 87.49
Epoch 0023  Loss 0.44  Accuracy 88.02
Epoch 0024  Loss 0.43  Accuracy 87.99
Epoch 0025  Loss 0.42  Accuracy 88.31
Epoch 0026  Loss 0.42  Accuracy 88.53
Epoch 0027  

## Training the ConvNet Model

In [ ]:
train_model(cnn_model)

Epoch 0001  Loss 0.27  Accuracy 54.75
Epoch 0002  Loss 0.05  Accuracy 65.24
Epoch 0003  Loss 0.03  Accuracy 92.04
Epoch 0004  Loss 0.03  Accuracy 93.29
Epoch 0005  Loss 0.02  Accuracy 96.63
Epoch 0006  Loss 0.02  Accuracy 97.42
Epoch 0007  Loss 0.01  Accuracy 89.57
Epoch 0008  Loss 0.01  Accuracy 97.12
Epoch 0009  Loss 0.01  Accuracy 97.12
Epoch 0010  Loss 0.01  Accuracy 97.28
Epoch 0011  Loss 0.01  Accuracy 97.85
Epoch 0012  Loss 0.01  Accuracy 97.27
Epoch 0013  Loss 0.01  Accuracy 98.66
Epoch 0014  Loss 0.00  Accuracy 99.11
Epoch 0015  Loss 0.00  Accuracy 98.44
Epoch 0016  Loss 0.00  Accuracy 98.21
Epoch 0017  Loss 0.00  Accuracy 99.12
Epoch 0018  Loss 0.00  Accuracy 99.12
Epoch 0019  Loss 0.00  Accuracy 99.05
Epoch 0020  Loss 0.00  Accuracy 99.16
Epoch 0021  Loss 0.00  Accuracy 99.24
Epoch 0022  Loss 0.00  Accuracy 99.22
Epoch 0023  Loss 0.00  Accuracy 99.09
Epoch 0024  Loss 0.00  Accuracy 99.39
Epoch 0025  Loss 0.00  Accuracy 99.40
Epoch 0026  Loss 0.00  Accuracy 99.45
Epoch 0027  

## Training with PyTorch data processing API

One of the pain points for ML researchers/practioners when building a new ML model is the data processing. Here, we demonstrate how to use data processing API of [PyTorch](https://pytorch.org/) to train a model with Objax. Different deep learning library comes with different data processing APIs, and depending on your preference, you can choose an API and easily combine with Objax.

Similarly, we prepare an `MNIST` dataset and apply the same data preprocessing.

In [ ]:
import torch
from torchvision import datasets, transforms

transform=transforms.Compose([
                              transforms.Pad((2,2,2,2), 0),
                              transforms.ToTensor(),
                              transforms.Lambda(lambda x: np.concatenate([x] * 3, axis=0)),
                              transforms.Lambda(lambda x: x * 2 - 1)
                              ])
train_dataset = datasets.MNIST(os.environ['HOME'], train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(os.environ['HOME'], train=False, download=True, transform=transform)

# Define data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch)

We replace data processing pipeline of the train and test loop with `train_loader` and `test_loader` and that's it!

In [ ]:
# Train loop

def train_model_with_torch_data_api(model):

  def predict(model, x):
    """""" 
    return objax.functional.softmax(model(x,  training=False))
    
  def flatten_image(x):
    """Flatten the image before passing it to the DNN."""
    if isinstance(model, DNNet):
      return objax.functional.flatten(x)
    else:
      return x
  
  opt = objax.optimizer.Momentum(model.vars())

  # Cross Entropy Loss
  def loss(x, label):
    return objax.functional.loss.cross_entropy_logits_sparse(model(x, training=True), label).mean()

  gv = objax.GradValues(loss, model.vars())
  def train_op(x, label):
    g, v = gv(x, label)  # returns gradients, loss
    opt(lr, g)
    return v

  train_op = objax.Jit(train_op, gv.vars() + opt.vars())  
  
  for epoch in range(epochs):
    avg_loss = 0
    tot_data = 0
    for _, (img, label) in enumerate(train_loader):
      avg_loss += float(train_op(flatten_image(img.numpy()), label.numpy())[0]) * len(img)
      tot_data += len(img)
    avg_loss /= tot_data

    # Eval
    accuracy = 0
    tot_data = 0
    for _, (img, label) in enumerate(test_loader):
      accuracy += (np.argmax(predict(model, flatten_image(img.numpy())), axis=1) == label.numpy()).sum()
      tot_data += len(img)
    accuracy /= tot_data
    print('Epoch %04d  Loss %.2f  Accuracy %.2f' % (epoch + 1, avg_loss, 100 * accuracy))

## Training the DNN Model with PyTorch data API

In [ ]:
dnn_layer_sizes = 3072, 128, 10
dnn_model = DNNet(dnn_layer_sizes, objax.functional.leaky_relu)
train_model_with_torch_data_api(dnn_model)

## Training the ConvNet Model with PyTorch data API

In [ ]:
cnn_model = ConvNet(nin=3, nclass=10)
cnn_model.vars().print()
train_model_with_torch_data_api(cnn_model)

## What's Next

We have learned how to use existing models and define new models to classify MNIST. Next, you can read one or more of the in-depth topics or browse through the Objax's APIs.